<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [12]:
import math
import numpy as np
import pandas as pd

In [5]:
# test
financial_statement = {
    'net_income': 5,
    'revenue': 2,
    'current_assets': 3,
    'current_liabilities': 3,
    'inventory': 1,
    'total_liabilities': 1,
    'total_equity': 1,
    'total_assets': 10,
    'profit_before_tax': 1,
    'interest_expense':1,
    'shareholders_equity':10,
    'capital_expenditures':3,
    'earnings_per_share': 0.5,
    'dividend': 0.5
}

In [6]:
# need the latest price_per_share, total_shares_outstanding, dividend maybe using API??
class fundamental_analysis:
    def __init__(self, data, price_per_share, total_shares_outstanding):
        self.data = data
        self.price_per_share = price_per_share
        self.total_shares_outstanding = total_shares_outstanding
        
        # Analyze Liquidity
        self.current_ratio = self.data['current_assets'] / self.data['current_liabilities']
        self.quick_ratio = (self.data['current_assets'] - self.data['inventory'])/ self.data['current_liabilities']
        
        # Assess Leverage
        self.debt_to_equity_ratio = self.data['total_liabilities'] / self.data['total_equity']
        self.interest_coverage_ratio = self.data['profit_before_tax'] / self.data['interest_expense']
        
        # Evaluate Profitability
        self.net_profit_margin = self.data['net_income']/self.data['revenue']
        self.return_on_assets = self.data['net_income']/self.data['total_assets']
        self.return_on_equity = self.data['net_income']/self.data['shareholders_equity']
        
        # Assess Cash Flow
        self.operating_cash_flow = self.data['net_income'] + self.data['interest_expense'] + self.data['current_assets'] - self.data['current_liabilities'] - self.data['inventory']
        self.free_cash_flow = self.operating_cash_flow - self.data['capital_expenditures']
        
        # Examine Solvency
        self.debt_to_asset_ratio = self.data['total_liabilities'] / self.data['total_assets']
        self.equity_ratio = self.data['total_equity'] / self.data['total_assets']
        
        # Efficiency Ratio
        self.asset_turnover_ratio = self.data['revenue'] / self.data['total_assets']

        # Market Valuation Ratios
        self.pe_ratio = self.price_per_share / self.data['earnings_per_share']
        self.pb_ratio = self.price_per_share * self.total_shares_outstanding / self.data['total_equity']
        self.dividend_yield = self.data['dividend'] / self.price_per_share
        
    def Altman_Z_Score(self, previous_retained_earning):
        '''
        Financial model used to predict the likelihood of a company going brankrupt.
        Z > 2.99: The company is considered safe from brankruptcy.
        1.81 < Z < 2.99: The company is in the gray zone, meaning there is a moderate risk of bankruptcy
        Z < 1.81: The company is in the distress zone, which indicates a high risk of bankruptcy.
        '''
        self.previous_retained_earning = previous_retained_earning
        x1 = (self.data['current_assets'] - self.data['current_liabilities']) / self.data['total_assets']
        x2 = self.previous_retained_earning / self.data['total_assets']
        x3 = self.data['net_income'] / self.data['total_assets']
        x4 = (self.price_per_share * self.total_shares_outstanding) / self.data['total_liabilities']
        x5 = self.data['revenue'] / self.data['total_assets']
        
        z = 1.2 * x1 + 1.4 * x2 + 3.3 * x3 + 0.6 * x4 + 1.0 * x5
        if z > 2.99:
            return print(f"Z-Score:{z:.3f} - Safe Zone")
        elif 1.81 < Z <= 2.99:
            return print(f"Z-Score:{z:.3f} - Grey Zone")
        else:
            return print(f"Z-Score:{z:.3f} - Distress Zone")
    
    def Ohlson_O_Score(self, depreciation=0, amortization=0, gains_on_sales=0, losses_on_sales=0):
        '''
        The Ohlson O-Score is a bankruptcy prediction model developed by James Ohlson in 1980, which assesses 
        the probability that a firm will go bankrupt. It uses financial ratios and data to calculate a score 
        that helps predict bankruptcy risk. The lower the O-Score, the lower the probability of bankruptcy.
        '''
        x1 = 0.407 * math.log(self.data['total_assets'])
        x2 = 6.03 * self.data['total_liabilities'] / self.data['total_assets']
        x3 = 1.43 * (self.data['current_assets'] - self.data['current_liabilities']) / self.data['total_assets']
        x4 = 0.0757 * self.data['current_liabilities'] / self.data['current_assets']
        x5 = 2.37 * self.data['net_income'] / self.data['total_assets']
        x6 = 1.83 * (self.data['net_income'] + depreciation + amortization + gains_on_sales - losses_on_sales) / self.data['total_liabilities']
        x7 = 0.521 * (self.data['net_income'] - self.data['dividend']) / self.data['total_assets']
        
        o_score = -1.32 - x1 + x2 - x3 + x4 - x5 - x6 -x7
        if o_score < -1.0:
            return print(f"O-Score: {o_score:.3f} - Distress Zone")
        elif -1.0 <= o_score <= 0.5:
            return print(f"O-Score: {o_score:.3f} - Grey Zone")
        else:
            return print(f"O-Score: {o_score:.3f} - Safe Zone")
        
    def Springate_S_Score(self, depreciation=0):
        '''
        The Springate Model (S-Score) is a financial distress prediction model that uses a company's financial 
        ratios to assess its likelihood of going bankrupt.
        '''
        a = (self.data['net_income'] + depreciation) / self.data['total_assets']
        b = (self.data['current_assets'] - self.data['current_liabilities']) / self.data['total_assets']
        c = self.data['revenue'] / self.data['total_assets']
        d = self.data['total_equity'] / self.data['total_liabilities']
        
        s_score = 1.03*a + 3.07*b + 0.66*c + 0.4*d
        if s_score < 0.5:
            return print(f"S-Score: {s_score:.3f} - Distress Zone")
        elif 0.5 <= s_score <= 1.0:
            return print(f"S-Score: {s_score:.3f} - Grey Zone")
        else:
            return print(f"S-Score: {s_score:.3f} - Safe Zone")
        
    def CAPM(self, stock, stock_monthly, index_monthly):
        stock_dif = stock['Price'][::-1].reset_index(drop=True).diff()
        stock_monthly_dif = stock_monthly['Price'][::-1].reset_index(drop=True).diff()
        index_monthly_dif = index_monthly['Price'][::-1].reset_index(drop=True).str.replace(',', '').astype(float).diff()

        stock_monthly_table = stock_monthly_dif[1:].to_frame(name='Stock Monthly')
        stock_table = stock_dif[1:].to_frame(name='Stock Daily')
        index_monthly_table = index_monthly_dif[1:].to_frame(name='Index Monthly')
        
        mean = stock_monthly_table.values.mean()
        var = stock_monthly_table.values.var()
        stock_monthly_table.loc['mean'] = mean
        stock_monthly_table.loc['var'] = var

        mean = stock_table.values.mean()
        var = stock_table.values.var()
        stock_table.loc['mean'] = mean
        stock_table.loc['var'] = var

        mean = index_monthly_table.values.mean()
        var = index_monthly_table.values.var()
        index_monthly_table.loc['mean'] = mean
        index_monthly_table.loc['var'] = var
        
        cov = np.cov(index_monthly_table.iloc[:10,0].values, stock_monthly_table.iloc[:10,0].values)[1,0]
        beta = cov / index_monthly_table.loc['var'][0]

        risk_free = 0.04 # 16th Oct 2024
        sharpe_ratio = (stock_monthly_table.loc['mean'][0] - risk_free) / np.sqrt(stock_monthly_table.loc['var'][0])

        treynor = (stock_monthly_table.loc['mean'][0] - risk_free) / beta
        
        print(f"Sharpe Ratio: {sharpe_ratio:.3f}") 
        if sharpe_ratio > 1:
            print("Good return per unit of risk.")
        elif 1>=sharpe_ratio>0:
            print("Return is not sufficient to compensate for the risk, suggesting that the investment may not be worthwhile compared to the risk-free rate.")
        else:
            print("Investment is performing worse than the risk-free rate.")

        print(f"\nTreynor: {treynor:.3f}") 
        if treynor > 0:
            print("The return is providing a return that exceeds the risk-free rate per unit of market risk.")
        else:
            print("The return is underperforming relative to the risk-free rate.")


In [7]:
test = fundamental_analysis(financial_statement, price_per_share=1, total_shares_outstanding=100)

In [8]:
# test.dividend_yield
test.Altman_Z_Score(previous_retained_earning = 5)

Z-Score:62.550 - Safe Zone


In [9]:
test.Ohlson_O_Score()

O-Score: -12.148 - Distress Zone


In [10]:
test.Springate_S_Score()

S-Score: 1.047 - Safe Zone


In [13]:
stock = pd.read_csv('Maybank Historical Data.csv')
stock_monthly = pd.read_csv('Maybank Historical Data (Monthly).csv')
index_monthly = pd.read_csv('FTSE Malaysia KLCI Historical Data (Monthly).csv')

In [14]:
test.CAPM(stock, stock_monthly, index_monthly)

Sharpe Ratio: 0.016
Return is not sufficient to compensate for the risk, suggesting that the investment may not be worthwhile compared to the risk-free rate.

Treynor: 0.717
The return is providing a return that exceeds the risk-free rate per unit of market risk.
